In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from typing import List, Optional, Tuple, Union

import math
import random

import time
import json

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
import gc
import os
import tempfile
import warnings
from typing import Optional, Tuple, Union

import torch
from torch import nn
from torch.nn import CrossEntropyLoss


from transformers.configuration_utils import PretrainedConfig
from transformers.modeling_outputs import BaseModelOutput, Seq2SeqLMOutput
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import add_start_docstrings, add_start_docstrings_to_model_forward, logging, replace_return_docstrings
from transformers.models.auto.configuration_auto import AutoConfig
from transformers.models.auto.modeling_auto import AutoModel, AutoModelForCausalLM
from transformers.models.encoder_decoder.configuration_encoder_decoder import EncoderDecoderConfig

from transformers.models.encoder_decoder.modeling_encoder_decoder import DEPRECATION_WARNING, shift_tokens_right


def forward_encoder_decoder(
    self,
    input_ids: Optional[torch.LongTensor] = None,
    token_importances: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.FloatTensor] = None,
    decoder_input_ids: Optional[torch.LongTensor] = None,
    decoder_attention_mask: Optional[torch.BoolTensor] = None,
    encoder_outputs: Optional[Tuple[torch.FloatTensor]] = None,
    past_key_values: Tuple[Tuple[torch.FloatTensor]] = None,
    inputs_embeds: Optional[torch.FloatTensor] = None,
    decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
    labels: Optional[torch.LongTensor] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
    **kwargs,
) -> Union[Tuple, Seq2SeqLMOutput]:
    r"""
    Returns:
    Examples:
    ```python
    >>> from transformers import EncoderDecoderModel, BertTokenizer
    >>> import torch
    >>> tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    >>> model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    ...     "bert-base-uncased", "bert-base-uncased"
    ... )  # initialize Bert2Bert from pre-trained checkpoints
    >>> # training
    >>> model.config.decoder_start_token_id = tokenizer.cls_token_id
    >>> model.config.pad_token_id = tokenizer.pad_token_id
    >>> model.config.vocab_size = model.config.decoder.vocab_size
    >>> input_ids = tokenizer("This is a really long text", return_tensors="pt").input_ids
    >>> labels = tokenizer("This is the corresponding summary", return_tensors="pt").input_ids
    >>> outputs = model(input_ids=input_ids, labels=labels)
    >>> loss, logits = outputs.loss, outputs.logits
    >>> # save and load from pretrained
    >>> model.save_pretrained("bert2bert")
    >>> model = EncoderDecoderModel.from_pretrained("bert2bert")
    >>> # generation
    >>> generated = model.generate(input_ids)
    ```"""

    return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    kwargs_encoder = {argument: value for argument, value in kwargs.items() if not argument.startswith("decoder_")}

    kwargs_decoder = {
        argument[len("decoder_") :]: value for argument, value in kwargs.items() if argument.startswith("decoder_")
    }

    if encoder_outputs is None:
        encoder_outputs = self.encoder(
            input_ids=input_ids,
            token_importances=token_importances,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            **kwargs_encoder,
        )
    elif isinstance(encoder_outputs, tuple):
        encoder_outputs = BaseModelOutput(*encoder_outputs)

    encoder_hidden_states = encoder_outputs[0]

    # optionally project encoder_hidden_states
    if (
        self.encoder.config.hidden_size != self.decoder.config.hidden_size
        and self.decoder.config.cross_attention_hidden_size is None
    ):
        encoder_hidden_states = self.enc_to_dec_proj(encoder_hidden_states)

    if (labels is not None) and (decoder_input_ids is None and decoder_inputs_embeds is None):
        decoder_input_ids = shift_tokens_right(
            labels, self.config.pad_token_id, self.config.decoder_start_token_id
        )

    # Decode
    decoder_outputs = self.decoder(
        input_ids=decoder_input_ids,
        attention_mask=decoder_attention_mask,
        encoder_hidden_states=encoder_hidden_states,
        encoder_attention_mask=attention_mask,
        inputs_embeds=decoder_inputs_embeds,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        use_cache=use_cache,
        past_key_values=past_key_values,
        return_dict=return_dict,
        **kwargs_decoder,
    )

    # Compute loss independent from decoder (as some shift the logits inside them)
    loss = None
    if labels is not None:
        warnings.warn(DEPRECATION_WARNING, FutureWarning)
        logits = decoder_outputs.logits if return_dict else decoder_outputs[0]
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.reshape(-1, self.decoder.config.vocab_size), labels.view(-1))

    if not return_dict:
        if loss is not None:
            return (loss,) + decoder_outputs + encoder_outputs
        else:
            return decoder_outputs + encoder_outputs

    return Seq2SeqLMOutput(
        loss=loss,
        logits=decoder_outputs.logits,
        past_key_values=decoder_outputs.past_key_values,
        decoder_hidden_states=decoder_outputs.hidden_states,
        decoder_attentions=decoder_outputs.attentions,
        cross_attentions=decoder_outputs.cross_attentions,
        encoder_last_hidden_state=encoder_outputs.last_hidden_state,
        encoder_hidden_states=encoder_outputs.hidden_states,
        encoder_attentions=encoder_outputs.attentions,
    )

In [5]:
import math
from typing import List, Optional, Tuple, Union

import torch
import torch.utils.checkpoint
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.activations import ACT2FN, gelu
from transformers.modeling_outputs import (
    BaseModelOutputWithPastAndCrossAttentions,
    BaseModelOutputWithPoolingAndCrossAttentions,
    CausalLMOutputWithCrossAttentions,
    MaskedLMOutput,
    MultipleChoiceModelOutput,
    QuestionAnsweringModelOutput,
    SequenceClassifierOutput,
    TokenClassifierOutput,
)
from transformers.modeling_utils import PreTrainedModel
from transformers.pytorch_utils import apply_chunking_to_forward, find_pruneable_heads_and_indices, prune_linear_layer
from transformers.utils import (
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)
from transformers.models.roberta.configuration_roberta import RobertaConfig

def forward_encoder(
    self,
    input_ids: Optional[torch.Tensor] = None,
    token_importances: Optional[torch.LongTensor] = None,
    attention_mask: Optional[torch.Tensor] = None,
    token_type_ids: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.Tensor] = None,
    head_mask: Optional[torch.Tensor] = None,
    inputs_embeds: Optional[torch.Tensor] = None,
    encoder_hidden_states: Optional[torch.Tensor] = None,
    encoder_attention_mask: Optional[torch.Tensor] = None,
    past_key_values: Optional[List[torch.FloatTensor]] = None,
    use_cache: Optional[bool] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:
    r"""
    encoder_hidden_states  (`torch.FloatTensor` of shape `(batch_size, sequence_length, hidden_size)`, *optional*):
        Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
        the model is configured as a decoder.
    encoder_attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, *optional*):
        Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
        the cross-attention if the model is configured as a decoder. Mask values selected in `[0, 1]`:
        - 1 for tokens that are **not masked**,
        - 0 for tokens that are **masked**.
    past_key_values (`tuple(tuple(torch.FloatTensor))` of length `config.n_layers` with each tuple having 4 tensors of shape `(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
        Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.
        If `past_key_values` are used, the user can optionally input only the last `decoder_input_ids` (those that
        don't have their past key value states given to this model) of shape `(batch_size, 1)` instead of all
        `decoder_input_ids` of shape `(batch_size, sequence_length)`.
    use_cache (`bool`, *optional*):
        If set to `True`, `past_key_values` key value states are returned and can be used to speed up decoding (see
        `past_key_values`).
    """
    output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
    output_hidden_states = (
        output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
    )
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    if self.config.is_decoder:
        use_cache = use_cache if use_cache is not None else self.config.use_cache
    else:
        use_cache = False

    if input_ids is not None and inputs_embeds is not None:
        raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
    elif input_ids is not None:
        input_shape = input_ids.size()
    elif inputs_embeds is not None:
        input_shape = inputs_embeds.size()[:-1]
    else:
        raise ValueError("You have to specify either input_ids or inputs_embeds")

    batch_size, seq_length = input_shape
    device = input_ids.device if input_ids is not None else inputs_embeds.device

    # past_key_values_length
    past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

    if attention_mask is None:
        attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)

    if token_type_ids is None:
        if hasattr(self.embeddings, "token_type_ids"):
            buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
            buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
            token_type_ids = buffered_token_type_ids_expanded
        else:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

    # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
    # ourselves in which case we just need to make it broadcastable to all heads.
    extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape)

    # If a 2D or 3D attention mask is provided for the cross-attention
    # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
    if self.config.is_decoder and encoder_hidden_states is not None:
        encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
        encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
        if encoder_attention_mask is None:
            encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
        encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
    else:
        encoder_extended_attention_mask = None

    # Prepare head mask if needed
    # 1.0 in head_mask indicate we keep the head
    # attention_probs has shape bsz x n_heads x N x N
    # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
    # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
    head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

    embedding_output = self.embeddings(
        input_ids=input_ids,
        position_ids=position_ids,
        token_type_ids=token_type_ids,
        inputs_embeds=inputs_embeds,
        past_key_values_length=past_key_values_length,
    )

    embedding_output = self.linear( torch.concat([embedding_output,token_importances],2) )
    
    encoder_outputs = self.encoder(
        embedding_output,
        attention_mask=extended_attention_mask,
        head_mask=head_mask,
        encoder_hidden_states=encoder_hidden_states,
        encoder_attention_mask=encoder_extended_attention_mask,
        past_key_values=past_key_values,
        use_cache=use_cache,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )
    sequence_output = encoder_outputs[0]
    pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

    if not return_dict:
        return (sequence_output, pooled_output) + encoder_outputs[1:]

    return BaseModelOutputWithPoolingAndCrossAttentions(
        last_hidden_state=sequence_output,
        pooler_output=pooled_output,
        past_key_values=encoder_outputs.past_key_values,
        hidden_states=encoder_outputs.hidden_states,
        attentions=encoder_outputs.attentions,
        cross_attentions=encoder_outputs.cross_attentions,
    )

In [6]:
from transformers import EncoderDecoderModel, AutoTokenizer

model_name = 'distilroberta-base'
model=EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name)

tokenizer=AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['roberta.encoder.layer.3.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.key.bias', 'roberta.encoder.layer.4.crossattention.self.ke

In [7]:
model.encoder.config.hidden_size

768

In [8]:
import types
funcType = type(model.forward)
model.forward = types.MethodType(forward_encoder_decoder, model)

import types
funcType = type(model.encoder.forward)
model.encoder.forward = types.MethodType(forward_encoder, model.encoder)



In [9]:
L=nn.Linear(model.encoder.config.hidden_size +1, model.encoder.config.hidden_size)

L.weight=torch.nn.parameter.Parameter(torch.eye(L.weight.shape[1], dtype=L.weight.dtype)[:L.weight.shape[0]])
L.bias=torch.nn.parameter.Parameter(torch.zeros(L.bias.shape, dtype=L.bias.dtype))

model.encoder.linear = L

# Test modified model

In [10]:
import os
import urllib.request
from tqdm import tqdm

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)
        
def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_data(data_path, url_path, suffix):    
    if not os.path.exists(data_path):
        os.makedirs(data_path)
        
    data_path = os.path.join(data_path, f'{suffix}.json')

    if not os.path.exists(data_path):
        print(f"Downloading CoQA {suffix} data split... (it may take a while)")
        download_url(url=url_path, output_path=data_path)
        print("Download completed!")

In [11]:
# Train data
train_url = "https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
download_data(data_path='coqa', url_path=train_url, suffix='train')

# Test data
test_url = "https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"
download_data(data_path='coqa', url_path=test_url, suffix='test')  # <-- Why test? See next slides for an answer!

In [12]:
with open(os.path.join('coqa', 'train.json'), 'r') as j:
    train = json.loads(j.read())

with open(os.path.join('coqa', 'test.json'), 'r') as j:
    test = json.loads(j.read())

In [13]:
train = train['data']
test = test['data']

In [14]:
lengths=[len(doc['questions']) for doc in train]

In [15]:
le=np.cumsum(np.array(lengths,dtype=np.float32))
train_end=np.where((le/le[-1])>0.8)[0][0]

validation = train[train_end : ] 
train = train[ : train_end]

In [16]:
print(len(train))
print(len(validation))

5771
1428


In [17]:
len_train=np.sum([len(doc['questions']) for doc in train])
len_val=np.sum([len(doc['questions']) for doc in validation])

len_tot=len_train+len_val
print(len_train,len_train/len_tot)
print(len_val,len_val/len_tot)

86909 0.7999208445700295
21738 0.20007915542997046


In [18]:
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, data, return_history=False):

        self.story=[d['story'] for d in data]
        self.questions=[d['questions'] for d in data]
        self.answers=[d['answers'] for d in data]
        lengths = [len(doc['questions']) for doc in data]
        self.lengths = np.cumsum(np.array(lengths,dtype=np.int32))
        self.R_H=return_history
        

    def __len__(self):
        return self.lengths[-1]

    def __getitem__(self, idx):
        f_idx=int(np.where(self.lengths > idx)[0][0])
        if f_idx>0:
            q_idx=idx-self.lengths[f_idx-1]
        else:
            q_idx=idx

        passage=self.story[f_idx]
        questions=self.questions[f_idx]
        answers=self.answers[f_idx]
        question=questions[q_idx]['input_text']
        answer=answers[q_idx]['input_text']
        span_start=int(answers[q_idx]['span_start'])
        span_end=int(answers[q_idx]['span_end'])
        span_text=answers[q_idx]['span_text']

        if self.R_H:
            history = np.concatenate([ [questions[i]['input_text'], answers[i]['input_text']] for i in range(q_idx)],0)
            return (passage,question,history), (span_start,span_end)

        return (passage,question), answer

In [19]:
from torch.utils.data import DataLoader

batch_size=16

train_dataloader = DataLoader(CustomImageDataset(train), batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(CustomImageDataset(validation), batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(CustomImageDataset(test), batch_size=batch_size, shuffle=True)

In [20]:
def train(model, tokenizer, n_epochs=3, learning_rate=1e-4):
    model.to('cuda')

    L=[]

    model.config.decoder_start_token_id = tokenizer.cls_token_id
    model.config.pad_token_id = tokenizer.pad_token_id

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    
    for epoch in range(n_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        optimizer.zero_grad()
        start_time = time.time()
        for i, data in enumerate(train_dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            (passage, question), answer = data

            # text_input = [question[i] + ' [SEP] ' + passage[i] for i in range(len(passage))]

            # zero the parameter gradients
            

            inputs = tokenizer(
                question,
                passage,
                padding=True,
                max_length=512,
                truncation=True,
                return_tensors="pt",
            ).input_ids

            labels = tokenizer(
                answer,
                max_length=512,
                truncation=True,
                padding=True,
                return_tensors="pt",
            ).input_ids

            #X=torch.tensor(input_ids,device='cuda')
            #y=torch.tensor(labels,device='cuda')
            
            #print(X.shape,y.shape)
            
            #if X.shape[1]>500:
            #    continue

            # the forward function automatically creates the correct decoder_input_ids
            outputs = model(input_ids=inputs.to('cuda'), labels=labels.to('cuda'),token_importances=torch.ones(inputs.shape[:2]+(1,)).to('cuda'))
            loss = outputs.loss
            loss.backward()

            if i%2**1==2**1-1:
                optimizer.step()
                optimizer.zero_grad()

                if i%2**10==i%2**10-1:
                    torch.cuda.empty_cache()


            # print statistics
            running_loss += loss.item()

            L.append(loss.detach().cpu().numpy())
            
            epoch_time = time.time() - start_time
            batch_time = epoch_time/(i+1)
            
            print(f"epoch: {epoch + 1}/{n_epochs}, {i + 1}/{len(train_dataloader)}, {epoch_time:.0f}s {batch_time*1e3:.0f}ms/step, lr: {optimizer.param_groups[0]['lr']:.3g}, loss: {running_loss/(i+1):.3g}")

        print(f"epoch: {epoch + 1}/{n_epochs}, {i + 1}/{len(train_dataloader)}, {epoch_time:.0f}s {batch_time*1e3:.0f}ms/step, lr: {optimizer.param_groups[0]['lr']:.3g}, loss: {running_loss/(i+1):.3g}")

    print('Finished Training')

In [21]:
train(model,tokenizer)

<ipython-input-4-4bc0e1d26dbb>:120: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


epoch: 1/3, 1/5432, 3s 2659ms/step, lr: 0.0001, loss: 12.9
epoch: 1/3, 2/5432, 4s 1829ms/step, lr: 0.0001, loss: 12.5
epoch: 1/3, 3/5432, 5s 1544ms/step, lr: 0.0001, loss: 12.9
epoch: 1/3, 4/5432, 6s 1405ms/step, lr: 0.0001, loss: 12.9
epoch: 1/3, 5/5432, 7s 1305ms/step, lr: 0.0001, loss: 12.4
epoch: 1/3, 6/5432, 7s 1235ms/step, lr: 0.0001, loss: 12.2
epoch: 1/3, 7/5432, 8s 1204ms/step, lr: 0.0001, loss: 11.9
epoch: 1/3, 8/5432, 9s 1161ms/step, lr: 0.0001, loss: 11.5
epoch: 1/3, 9/5432, 10s 1139ms/step, lr: 0.0001, loss: 11
epoch: 1/3, 10/5432, 11s 1123ms/step, lr: 0.0001, loss: 10.7
epoch: 1/3, 11/5432, 12s 1102ms/step, lr: 0.0001, loss: 10.3
epoch: 1/3, 12/5432, 13s 1096ms/step, lr: 0.0001, loss: 10
epoch: 1/3, 13/5432, 14s 1073ms/step, lr: 0.0001, loss: 9.74
epoch: 1/3, 14/5432, 15s 1068ms/step, lr: 0.0001, loss: 9.46
epoch: 1/3, 15/5432, 16s 1052ms/step, lr: 0.0001, loss: 9.34
epoch: 1/3, 16/5432, 17s 1049ms/step, lr: 0.0001, loss: 9.2
epoch: 1/3, 17/5432, 18s 1044ms/step, lr: 0.00

KeyboardInterrupt: ignored

In [23]:
model.encoder.linear.weight[:,-1]

tensor([ 4.5172e-04, -1.4164e-04,  4.5031e-05,  3.6363e-04,  5.5747e-04,
        -7.5718e-07,  7.3285e-05,  1.7333e-04, -2.1527e-05, -1.6153e-04,
         1.0575e-04,  2.7516e-04, -2.7098e-04, -1.9887e-04,  5.6152e-05,
         3.6800e-04, -2.4561e-04,  5.7147e-04,  3.1572e-04, -3.1121e-04,
         8.5146e-05,  3.1175e-04, -5.2163e-04, -3.7501e-04, -6.2943e-06,
        -5.7430e-04,  5.4382e-04,  4.3328e-04, -4.0453e-04, -6.9903e-04,
         6.5399e-04,  1.7184e-04,  5.4434e-04,  1.7659e-04,  3.6106e-04,
        -6.7700e-04, -4.0689e-04,  1.1316e-04, -9.5303e-05,  2.4923e-04,
         1.8745e-04,  1.1084e-04,  3.0432e-04,  3.2041e-04,  5.4546e-04,
         1.6255e-04, -2.0517e-04,  7.5383e-04, -4.9268e-04, -1.3895e-04,
        -3.6883e-04, -1.5878e-04, -4.1411e-04, -6.5409e-04,  2.1377e-04,
         5.3971e-04, -4.2485e-04,  2.9226e-04, -4.3067e-04, -1.6943e-05,
        -1.3083e-04,  5.6024e-04, -1.8348e-04,  7.6923e-04, -5.5457e-05,
         1.6870e-04, -4.3105e-04, -1.3601e-05,  2.2